# Product Recommendation with Feathr on Azure (Advanced)

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## 1. Prerequisites

### Use Azure Resource Manager (ARM) to Provision Azure Resources for Feathr

First step is to provision required cloud resources if you want to use Feathr. Feathr provides a python based client to interact with cloud resources.

Please follow the steps [here](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html) to provision required cloud resources. Due to the complexity of the possible cloud environment, it is almost impossible to create a script that works for all the use cases. Because of this, [azure_resource_provision.sh](https://github.com/feathr-ai/feathr/blob/main/docs/how-to-guides/azure_resource_provision.sh) is a full end to end command line to create all the required resources, and you can tailor the script as needed, while [the companion documentation](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-cli.html) can be used as a complete guide for using that shell script.

If you already have an existing resource group and only want to install few resources manually you can refer to the cli documentation [here](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-cli.html). It provides CLI commands to install the needed resources.

> Please Note: CLI documentation is for advance users since there are lot of configurations and role assignment that would have to be done manually. Therefore, ARM template is the preferred way to deploy.

The below architecture diagram represents how different resources interact with each other.
![Architecture](https://github.com/feathr-ai/feathr/blob/main/docs/images/architecture.png?raw=true)

### Set the required permissions

Before you proceed further, you would need additional permissions to:
* Access the keyvault,
* Access the Storage Blob as a Contributor, and
* Submit jobs to Synapse cluster.

Run the following commands in the [Cloud Shell](https://shell.azure.com) before moving forward. Please replace `YOUR_RESOURCE_PREFIX` with the value you used in ARM template deployment.

```
    resource_prefix="YOUR_RESOURCE_PREFIX"
    synapse_workspace_name="${resource_prefix}syws"
    keyvault_name="${resource_prefix}kv"
    objectId=$(az ad signed-in-user show --query id -o tsv)
    az keyvault update --name $keyvault_name --enable-rbac-authorization false
    az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
    az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
    az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
```


### Install Feathr python package and it's dependencies

Here, we install the package from the repository's main branch. To use the latest release, you may run `pip install feathr[notebook]` instead. If so, however, some of the new features in this notebook might not work.

In [ ]:
# Uncomment and run this cell to install feathr from the latest codes in the repo. You may use `pip install feathr[notebook]` as well.
# !pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]"  

In [ ]:
# We also install InterpretML package for model explainability
!pip install interpret

## 2. Config Feathr Client

In [ ]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame


import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df
from feathr.utils.platform import is_databricks


print(f"Feathr version: {feathr.__version__}")

In [ ]:
RESOURCE_PREFIX = ""  # TODO fill the value used to deploy the resources via ARM template
PROJECT_NAME = "product_recommendation"

# Currently support: 'azure_synapse', 'databricks', and 'local' 
SPARK_CLUSTER = "local"

# TODO fill values to use databricks cluster:
DATABRICKS_CLUSTER_ID = None             # Set Databricks cluster id to use an existing cluster
if is_databricks():
    # If this notebook is running on Databricks, its context can be used to retrieve token and instance URL
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    DATABRICKS_WORKSPACE_TOKEN_VALUE = ctx.apiToken().get()
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = f"https://{ctx.tags().get('browserHostName').get()}"
else:
    DATABRICKS_WORKSPACE_TOKEN_VALUE = None                  # Set Databricks workspace token to use databricks
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = None  # Set Databricks workspace url to use databricks

# TODO fill values to use Azure Synapse cluster:
AZURE_SYNAPSE_SPARK_POOL = None  # Set Azure Synapse Spark pool name
ADLS_KEY = None                  # Set Azure Data Lake Storage key to use Azure Synapse

# TODO if you deployed resources manually using different names, you'll need to change the following values accordingly: 
ADLS_ACCOUNT = f"{RESOURCE_PREFIX}dls"
ADLS_FS_NAME = f"{RESOURCE_PREFIX}fs"
AZURE_SYNAPSE_URL = f"https://{RESOURCE_PREFIX}syws.dev.azuresynapse.net"  # Set Azure Synapse workspace url to use Azure Synapse
KEY_VAULT_URI = f"https://{RESOURCE_PREFIX}kv.vault.azure.net"
REDIS_HOST = f"{RESOURCE_PREFIX}redis.redis.cache.windows.net"
REGISTRY_ENDPOINT = f"https://{RESOURCE_PREFIX}webapp.azurewebsites.net/api/v1"
AZURE_SYNAPSE_WORKING_DIR = f"abfss://{ADLS_FS_NAME}@{ADLS_ACCOUNT}.dfs.core.windows.net/{PROJECT_NAME}"

# An existing Feathr config file path. If None, we'll generate a new config based on the constants in this cell.
FEATHR_CONFIG_PATH = None

USE_CLI_AUTH = False  # Set to True to use CLI authentication

# If set True, register the features to Feathr registry.
REGISTER_FEATURES = False

# (For the notebook test pipeline) If true, use ScrapBook package to collect the results.
SCRAP_RESULTS = False

### Get Azure credential to access Azure resources

In [ ]:
# Get an authentication credential to access Azure resources and register features
if USE_CLI_AUTH:
    # Use AZ CLI interactive browser authentication
    !az login --use-device-code
    from azure.identity import AzureCliCredential
    credential = AzureCliCredential(additionally_allowed_tenants=['*'],)
elif "AZURE_TENANT_ID" in os.environ and "AZURE_CLIENT_ID" in os.environ and "AZURE_CLIENT_SECRET" in os.environ:
    # Use Environment variable secret
    from azure.identity import EnvironmentCredential
    credential = EnvironmentCredential()
else:
    # Try to use the default credential
    from azure.identity import DefaultAzureCredential
    credential = DefaultAzureCredential(
        exclude_interactive_browser_credential=False,
        additionally_allowed_tenants=['*'],
    )

### Set the environment variables
Set the environment variables that will be used by Feathr as configuration. Feathr supports configuration via enviroment variables and yaml. You can read more about it [here](https://feathr-ai.github.io/feathr/how-to-guides/feathr-configuration-and-env.html).

In [ ]:
if SPARK_CLUSTER == "azure_synapse" and not os.environ.get("ADLS_KEY"):
    os.environ["ADLS_KEY"] = ADLS_KEY
elif SPARK_CLUSTER == "databricks" and not os.environ.get("DATABRICKS_WORKSPACE_TOKEN_VALUE"):
    os.environ["DATABRICKS_WORKSPACE_TOKEN_VALUE"] = DATABRICKS_WORKSPACE_TOKEN_VALUE

In [ ]:
# Redis password
if 'REDIS_PASSWORD' not in os.environ:
    from azure.keyvault.secrets import SecretClient
    
    secret_client = SecretClient(vault_url=KEY_VAULT_URI, credential=credential)
    retrieved_secret = secret_client.get_secret('FEATHR-ONLINE-STORE-CONN').value
    os.environ['REDIS_PASSWORD'] = retrieved_secret.split(",")[1].split("password=", 1)[1]

> If you run into issues where Key vault or other resources are not found through notebook despite being there, make sure you are connected to the right subscription by running the command: 'az account show' and 'az account set --subscription <subscription_id>'

### Generate the Feathr client configuration

The code below will write the onfiguration to a temporary location that will be used by a Feathr client. Please refer to [feathr_config.yaml](https://github.com/feathr-ai/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) for full list of configuration options and details.

In [ ]:
if FEATHR_CONFIG_PATH:
    config_path = FEATHR_CONFIG_PATH
else:
    config_path = generate_config(
        resource_prefix=RESOURCE_PREFIX,
        project_name=PROJECT_NAME,
        online_store__redis__host=REDIS_HOST,
        feature_registry__api_endpoint=REGISTRY_ENDPOINT,
        spark_config__spark_cluster=SPARK_CLUSTER,
        spark_config__azure_synapse__dev_url=AZURE_SYNAPSE_URL,
        spark_config__azure_synapse__pool_name=AZURE_SYNAPSE_SPARK_POOL,
        spark_config__azure_synapse__workspace_dir=AZURE_SYNAPSE_WORKING_DIR,
        spark_config__databricks__workspace_instance_url=SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL,
        databricks_cluster_id=DATABRICKS_CLUSTER_ID,
    )

with open(config_path, 'r') as f: 
    print(f.read())

### Initialize Feathr Client

In [ ]:
client = FeathrClient(config_path=config_path, credential=credential)

## 3. Prepare Datasets

1. Download datasets
2. Upload to cloud storage if necessary so that the target cluster can consume them as the data sources

In [ ]:
# Use dbfs if the notebook is running on Databricks
if is_databricks():
    WORKING_DIR = f"/dbfs/{PROJECT_NAME}"
else:
    WORKING_DIR = PROJECT_NAME

In [ ]:
# Download datasets
user_observation_file_path = f"{WORKING_DIR}/user_observation.csv"
user_profile_file_path = f"{WORKING_DIR}/user_profile.csv"
user_purchase_history_file_path = f"{WORKING_DIR}/user_purchase_history.csv"
product_detail_file_path = f"{WORKING_DIR}/product_detail.csv"
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    dst_filepath=user_observation_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    dst_filepath=user_profile_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    dst_filepath=user_purchase_history_file_path,
)
maybe_download(
    src_url=PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
    dst_filepath=product_detail_file_path,
)

In [ ]:
# Upload files to cloud if needed
if client.spark_runtime == "local":
    # In local mode, we can use the same data path as the source.
    # If the notebook is running on databricks, DATA_FILE_PATH should be already a dbfs path.
    user_observation_source_path = user_observation_file_path
    user_profile_source_path = user_profile_file_path
    user_purchase_history_source_path = user_purchase_history_file_path
    product_detail_source_path = product_detail_file_path
elif client.spark_runtime == "databricks" and is_databricks():
    # If the notebook is running on databricks, we can use the same data path as the source.
    user_observation_source_path = user_observation_file_path.replace("/dbfs", "dbfs:")
    user_profile_source_path = user_profile_file_path.replace("/dbfs", "dbfs:")
    user_purchase_history_source_path = user_purchase_history_file_path.replace("/dbfs", "dbfs:")
    product_detail_source_path = product_detail_file_path.replace("/dbfs", "dbfs:")
else:
    # Otherwise, upload the local file to the cloud storage (either dbfs or adls).
    user_observation_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_observation_file_path)
    user_profile_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_profile_file_path)
    user_purchase_history_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(user_purchase_history_file_path)
    product_detail_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(product_detail_file_path)

## 3. Define Sharable Features using Feathr API

### Understand raw datasets
We have three datasets to work with:
* Observation dataset (a.k.a. labeled dataset)
* User profile
* User purchase history
* Product details

In [ ]:
# Observation dataset
# Observation dataset usually comes with a event_timestamp to denote when the observation happened.
# The label here is product_rating. Our model objective is to predict a user's rating for this product.
pd.read_csv(user_observation_file_path).head()

In [ ]:
# User profile dataset
# Used to generate user features
pd.read_csv(user_profile_file_path).head()

In [ ]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to genearte features.
pd.read_csv(user_purchase_history_file_path).head()

In [ ]:
# Product detail dataset.
# Used to generate product features.
pd.read_csv(product_detail_file_path).head()

### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive. 

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields. 

In [ ]:
def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col

    return df.withColumn("tax_rate_decimal", col("tax_rate") / 100)


batch_source = HdfsSource(
    name="userProfileData",
    path=user_profile_source_path,
    preprocessing=feathr_udf_preprocessing,
)

In [ ]:
# Let's define some features for users so our recommendation can be customized for users.
user_id = TypedKey(
    key_column="user_id",
    key_column_type=ValueType.INT32,
    description="user id",
    full_name="product_recommendation.user_id",
)

feature_user_age = Feature(
    name="feature_user_age",
    key=user_id,
    feature_type=INT32,
    transform="age",
)
feature_user_tax_rate = Feature(
    name="feature_user_tax_rate",
    key=user_id,
    feature_type=FLOAT,
    transform="tax_rate_decimal",
)
feature_user_gift_card_balance = Feature(
    name="feature_user_gift_card_balance",
    key=user_id,
    feature_type=FLOAT,
    transform="gift_card_balance",
)
feature_user_has_valid_credit_card = Feature(
    name="feature_user_has_valid_credit_card",
    key=user_id,
    feature_type=BOOLEAN,
    transform="number_of_credit_cards > 0",
)

features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card,
]

user_feature_anchor = FeatureAnchor(
    name="anchored_features", source=batch_source, features=features
)

In [ ]:
# Let's define some features for the products so our recommendation can be customized for proudcts.
product_batch_source = HdfsSource(
    name="productProfileData",
    path=product_detail_source_path,
)

product_id = TypedKey(
    key_column="product_id",
    key_column_type=ValueType.INT32,
    description="product id",
    full_name="product_recommendation.product_id",
)

feature_product_quantity = Feature(
    name="feature_product_quantity",
    key=product_id,
    feature_type=FLOAT,
    transform="quantity",
)
feature_product_price = Feature(
    name="feature_product_price", key=product_id, feature_type=FLOAT, transform="price"
)

product_features = [feature_product_quantity, feature_product_price]

product_feature_anchor = FeatureAnchor(
    name="product_anchored_features",
    source=product_batch_source,
    features=product_features,
)

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

In [ ]:
purchase_history_data = HdfsSource(
    name="purchase_history_data",
    path=user_purchase_history_source_path,
    event_timestamp_column="purchase_date",
    timestamp_format="yyyy-MM-dd",
)

agg_features = [
    Feature(
        name="feature_user_avg_purchase_for_90days",
        key=user_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(purchase_amount)", agg_func="AVG", window="90d"
        ),
    )
]

user_agg_feature_anchor = FeatureAnchor(
    name="aggregationFeatures", source=purchase_history_data, features=agg_features
)

### Derived Features Section
Derived features are the features that are computed from other features. They could be computed from anchored features or other derived features.

In [ ]:
derived_features = [
    DerivedFeature(
        name="feature_user_purchasing_power",
        key=user_id,
        feature_type=FLOAT,
        input_features=[feature_user_gift_card_balance, feature_user_has_valid_credit_card],
        transform="feature_user_gift_card_balance + if(boolean(feature_user_has_valid_credit_card), 100, 0)",
    )
]

### Build features

Lastly, we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features which is not anchored to a source.

In [ ]:
client.build_features(
    anchor_list=[user_agg_feature_anchor, user_feature_anchor, product_feature_anchor],
    derived_feature_list=derived_features,
)

## 4. Create Training Data using Point-in-Time Correct Feature join

To create a training dataset using Feathr, we need to provide a **feature join settings** to specify what features and how these features should be joined to the observation data. 

Also note that since a `FeatureQuery` accepts features of the same join key, we define two query objects, one for `user_id` key and the other one for `product_id` and pass them together to compute offline features. 

To learn more on this topic, please refer to [Point-in-time Correctness document](https://feathr-ai.github.io/feathr/concepts/point-in-time-join.html).

In [ ]:
user_feature_query = FeatureQuery(
    feature_list=[feat.name for feat in features + agg_features + derived_features],
    key=user_id,
)

product_feature_query = FeatureQuery(
    feature_list=[feat.name for feat in product_features],
    key=product_id,
)

settings = ObservationSettings(
    observation_path=user_observation_source_path,
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd",
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=[user_feature_query, product_feature_query],
    output_path=user_profile_source_path.rpartition("/")[0] + f"/product_recommendation_features.avro",
)
client.wait_job_to_finish(timeout_sec=5000)

Let's use the helper function `get_result_df` to download the result and view it:

In [ ]:
res_df = get_result_df(client)
res_df.head()

### Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr. Here, we use **EBM (Explainable Boosting Machine)** regressor from [InterpretML](https://github.com/interpretml/interpret) package to visualize the modeling results.

In [ ]:
from interpret import show
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


# Fill None values with 0
final_df = (
    res_df
    .drop(["event_timestamp"], axis=1, errors="ignore")
    .fillna(0)
)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop(["product_rating"], axis=1),
    final_df["product_rating"].astype("float64"),
    test_size=0.2,
    random_state=42,
)

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

# Note, currently InterpretML's visualization dashboard doesn't work w/ VSCODE notebook viewer
# https://github.com/interpretml/interpret/issues/317
ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
# Predict and evaluate
y_pred = ebm.predict(X_test)
rmse = sqrt(mean_squared_error(y_test.values.flatten(), y_pred))

print(f"Root mean squared error: {rmse}")

## 5. Feature Materialization

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:

In [ ]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings(
    name="user_feature_setting",
    sinks=[redisSink],
    feature_names=["feature_user_age", "feature_user_gift_card_balance"],
)

client.materialize_features(settings=settings, allow_materialize_non_agg_feature=True)
client.wait_job_to_finish(timeout_sec=5000)

We can then get the features from the online store (Redis):

In [ ]:
client.get_online_features(
    "user_features", "2", ["feature_user_age", "feature_user_gift_card_balance"]
)

In [ ]:
client.multi_get_online_features(
    "user_features", ["1", "2"], ["feature_user_age", "feature_user_gift_card_balance"]
)

We can also materialize product features into a separate table.

In [ ]:
# Materialize product features
backfill_time = BackfillTime(
    start=datetime(2020, 5, 20),
    end=datetime(2020, 5, 20),
    step=timedelta(days=1),
)

redisSink = RedisSink(table_name="product_features")
settings = MaterializationSettings(
    "product_feature_setting",
    backfill_time=backfill_time,
    sinks=[redisSink],
    feature_names=["feature_product_price"],
)

client.materialize_features(settings, allow_materialize_non_agg_feature=True)
client.wait_job_to_finish(timeout_sec=5000)

In [ ]:
client.get_online_features("product_features", "2", ["feature_product_price"])

## 6. Feature Registration
Lastly, we can also register the features and share them across teams:

In [ ]:
if REGISTER_FEATURES:
    try:
        client.register_features()
    except Exception as e:
        print(e)
    print(client.list_registered_features(project_name=PROJECT_NAME))

## Cleanup

In [ ]:
# Cleaning up the output files. CAUTION: this maybe dangerous if you "reused" the project name.
import shutil
shutil.rmtree(WORKING_DIR, ignore_errors=False)

Scrap Variables for Unit-Test

In [ ]:
if SCRAP_RESULTS:
    # Record results for test pipelines
    import scrapbook as sb
    sb.glue(
        "user_features",
        client.get_online_features(
            "user_features", "2", ["feature_user_age", "feature_user_gift_card_balance"]
        ),
    )
    sb.glue(
        "product_features",
        client.get_online_features(
            "product_features", "2", ["feature_product_price"]
        ),
    )
    
    sb.glue("rmse", rmse)